In [1]:
%%bash
chia version
cdv --version

1.4.0
cdv, version 1.0.8


# Tic Tac Toe

Inner Puzzle is [tic tac toe coin](tic-tac-toe-coin.ipynb).

In [2]:
from pathlib import Path
import sys

# import singleton_helpers and tic_tac_toe
sys.path.insert(0, ".")
import tic_tac_toe
sys.path.insert(0, "..")
import singleton_helpers

In [3]:
from chia.types.blockchain_format.program import Program
from clvm_tools.clvmc import compile_clvm_text
def load_program(file_path, search_paths):
    clsp = Path(file_path).read_text()
    return Program(
        compile_clvm_text(clsp, search_paths)
    )

tic_tac_toe_puzzle = load_program('tic-tac-toe.clsp', '.')
tic_tac_toe_coin_puzzle = load_program('tic-tac-toe-coin.clsp', '.')

In [8]:
from chia.wallet.puzzles import (singleton_top_layer, p2_delegated_puzzle_or_hidden_puzzle)
from cdv.test import Network, Wallet

network: Network = await Network.create()

# use as function parameters
get_coin_records_by_parent_ids = network.sim_client.get_coin_records_by_parent_ids
get_coin_record_by_name = network.sim_client.get_coin_record_by_name
get_puzzle_and_solution = network.sim_client.get_puzzle_and_solution        
get_block_records = network.sim_client.get_block_records
get_additions_and_removals = network.sim_client.get_additions_and_removals

await network.farm_block()

alice: Wallet = network.make_wallet("alice")
bob: Wallet = network.make_wallet("bob")
await network.farm_block(farmer=alice)
await network.farm_block(farmer=bob)

print(f'alice balance:\t\t{alice.balance()}')
print(f'alice puzzle hash:\t{alice.puzzle_hash}')
print(f'bob balance:\t\t{bob.balance()}')
print(f'bob puzzle hash:\t{bob.puzzle_hash}')

alice balance:		2000000000000
alice puzzle hash:	4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3
bob balance:		2000000000000
bob puzzle hash:	87908e3f85bf4b55c7e7709915c2ce97a1e6ec1d227e54a04dbfee6862d546a5


In [11]:
# prepare tic tac toe coin
# empty board
board = [0] * 9
# player one (x)
player = 1
curried_tic_tac_toe_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzle(tic_tac_toe_puzzle, board, player)
curried_tic_tac_toe_coin_puzzle = tic_tac_toe_coin_puzzle.curry(
    tic_tac_toe_coin_puzzle,
    alice.puzzle_hash, # player one (x)
    bob.puzzle_hash, # player two (o)
    tic_tac_toe_puzzle, 
    curried_tic_tac_toe_puzzle)

In [12]:
standard_txn_coin_wrapper = await alice.choose_coin(1_750_000_000_000)
standard_txn_coin = standard_txn_coin_wrapper.as_coin()
standard_txn_puzzle = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(alice.pk())

inner_puzzle = singleton_top_layer.adapt_inner_to_singleton(curried_tic_tac_toe_coin_puzzle)
standard_coin_message, coin_spends = singleton_helpers.get_create_singleton_coin_spends(
                    standard_txn_coin,
                    standard_txn_puzzle,
                    odd_amount = 1023, 
                    inner_puzzle = inner_puzzle,
                    keys_values = [
                        ("game", "tic tac toe"),
                        ("player one", "alice"),
                        ("player two", "bob")
                    ]
              )
standard_txn_coin_spend = coin_spends[0]
launcher_coin_spend = coin_spends[1]
print("\nstarting coin spend:")
singleton_helpers.print_json(standard_txn_coin_spend.to_json_dict())
print("\nlauncher coin spend:")
singleton_helpers.print_json(launcher_coin_spend.to_json_dict())
print(f'\nlauncher id: {launcher_coin_spend.coin.name()}')
print(f'\nstandard_coin_message: {standard_coin_message.hex()}')


starting coin spend:
{
    "coin": {
        "amount": 1750000000000,
        "parent_coin_info": "0xe3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000001",
        "puzzle_hash": "0x4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff0bffff01ff02ffff03ffff09ff05ffff1dff0bffff1effff0bff0bffff02ff06ffff04ff02ffff04ff17ff8080808080808080ffff01ff02ff17ff2f80ffff01ff088080ff0180ffff01ff04ffff04ff04ffff04ff05ffff04ffff02ff06ffff04ff02ffff04ff17ff80808080ff80808080ffff02ff17ff2f808080ff0180ffff04ffff01ff32ff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff06ffff04ff02ffff04ff09ff80808080ffff02ff06ffff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080ffff04ffff01b0a042c855d234578415254b7870b711fb25e8f85beaa4a66bd0673d394c761fa156406c2e3bb375d5b18766d2a12cc918ff018080",
    "solution": "0xff80ffff01ffff33ffa0eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9ff8203ff80ffff3dffa0